In [1]:
# 주어진 문서에서 '.','!','?' 기호만을 이용하여 문장을 분리한다.
def split_sentence (doc):
    sentence_list = []
    temp_sentence = ""
    for char in doc:
        #현재 문자가 '.','!','?'중의 하나인 경우, 현재까지의 문장을
        #리스트에 추가하고, 문장을 초기화한다.
        if char == '.' or char == '!' or char == '?':
            sentence_list.append(temp_sentence)
            temp_sentence = ""
        #그렇지 않은 경우, 현재 문자를 문장에 계속 추가한다.
        else:
            temp_sentence += char
    return sentence_list

print (split_sentence("hey~~~ there!hi, how are you?I am\n fine, and you? I am good, too."))

['hey~~~ there', 'hi, how are you', 'I am\n fine, and you', ' I am good, too']


In [2]:
# 문장에서 기호를 제거하고, 하나의 공백만을 이용하여 단어를 구별한다.
def remove_symbols (sentence):
    #1. str.isalpha()함수를 이용하여 영문 알파벳이 아닌
    #   모든 숫자 및 기호를 스페이스로 변환한다.
    new_sentence = ""
    for char in sentence:
        if char.isalpha():
            new_sentence += char
        else:
            new_sentence += " "
    #print(new_sentence)

    #2. str.join()함수를 이용하여 공백이 여러개 있는 경우,
    #   하나로 만들어 이를 리턴한다.
    trimmed_sentence = ' '.join(new_sentence.split())
    return trimmed_sentence

print (remove_symbols("I am\n fine"))

I am fine


In [3]:
# 주어진 filepath의 파일을 오픈하여 전체 문자열을 리턴한다.
def get_doc_str(filepath):
    f = open(filepath,'r')
    doc = f.readlines()

    doc_str = ""
    for line in doc:
        doc_str += line
    f.close()
    return doc_str
    
print(get_doc_str("d1.txt"))

hey~~~ there!hi, how are you now I am
fine, and you? I am good, too and I have never been better too.


In [4]:
# 주어진 단어 리스트에서 window 구간 (begin_idx, end_idx (exclusive)) 
# 사이의 단어 리스트를 리턴한다.
def get_window_str(d1_word_list, begin_idx, end_idx):
    d1_window = "" 
    i = begin_idx
    while i < end_idx:
        d1_window += d1_word_list[i] + " "
        i += 1
    return d1_window.strip()

print(get_window_str(['hi', 'how', 'are','you'], 0, 2))

hi how


In [14]:
# 두 문장을 비교하여 최소 단어 수(min_window_size) 이상의 표절 구문 리스트를 리턴한다.
def get_max_common(d1_sentence, d2_sentence, min_window_size):
    d1_sentence = d1_sentence.lower()
    d2_sentence = d2_sentence.lower()
    
    common_phrase_list = []
    d1_word_list = d1_sentence.split(' ')
    d1_window_str = "" #d1 문장의 연속된 구문(Phrase) 윈도우 (단어수 기준)
    begin = 0 #연속된 구문 윈도우의 시작점   
    
    #시작점을 (문장의 끝 – 최소표절단어수)까지 이동하면서 표절 검사
    while begin <= len(d1_word_list) - min_window_size:
        # 윈도우의 끝점을 업데이트된 시작점과 최소윈도우 크기를 합한 값으로 업데이트
        end = begin + min_window_size 
        d1_window_str = get_window_str(d1_word_list, begin, end)
        #found_flag = False # 현재 시작점을 기준으로 표절 구문을 찾았는 지 여부를 나타내는 flag
        
        # 현재 시작점을 기준으로 표절 구문을 찾은 경우
        if d2_sentence.find(d1_window_str) > -1: 
            #found_flag = True
            # 윈도우의 끝점을 문장의 끝까지 이동하면서 현재 시작점을 기준으로 최대 표절 구문을 탐색
            while end < len(d1_word_list):
                d1_window_str = get_window_str(d1_word_list, begin, end)
                if d2_sentence.find(d1_window_str) > -1:
                    end += 1
                else: # 현재 윈도우가 더 이상 표절 구문이 아닌 경우 break 
                    break  
            
            end -= 1 # 표절 구문보다 한칸 더 이동했으므로 한칸 뒤로 Back
            
            # 현재 윈도우를 표절 구문 리스트에 추가
            print("[%d, %d]: " %(begin,end), get_window_str(d1_word_list, begin, end))
            common_phrase_list.append(get_window_str(d1_word_list, begin, end))
        
            # 표절구문을 찾은 경우, 표절구문의 끝점에서 다시 검사 시작
            # (시작점을 단순히 1 증가시켜도, 해당 구문은 결국 현재 표절 구문의 부분 구문이기 때문) 
            begin = end
            
        else: # 현재 시작점을 기준으로 공통 구문을 찾지 못한 경우, 시작점을 한칸만 이동해서 다시 탐색
            begin += 1
    
    return common_phrase_list

print(get_max_common("The Jupyter Notebook is an open source web application that allows you to create and share documents", 
                     "The Jupyter Notebook is an open source web application which allows you to create and share documents", 4))

[0, 9]:  the jupyter notebook is an open source web application
[10, 16]:  allows you to create and share
['the jupyter notebook is an open source web application', 'allows you to create and share']


In [15]:
# Main (간단한 표절 검사 프로그램)
MIN_COMMON_WORDS = 4 # 표절 검사 시 기준이 되는 공통 단어 수

# 1. 비교할 두 텍스트 파일로부터 전체 문자열을 가져옴 
d1_str = get_doc_str("d1.txt")
d2_str = get_doc_str("d2.txt")
print("d1_doc: ", d1_str, "\n")
print("d2_doc: ", d2_str, "\n")

# 2. 마침표, 느낌표, 물음표를 기준으로 문장 단위로 Split 
d1_sentence_list = split_sentence(d1_str)
d2_sentence_list = split_sentence(d2_str)

# 3. d1 문장 리스트를 기준으로 d2 문장들을 2중 for문으로 반복하여,
# 모든 d1, d2문장 조합을 비교하면서 최대 공통 구문을 추출
common_phrase_list = []
for d1_sentence in d1_sentence_list:
    for d2_sentence in d2_sentence_list:
        d1_sentence = remove_symbols(d1_sentence)
        d2_sentence = remove_symbols(d2_sentence)
        print("d1: %s, d2: %s" %(d1_sentence,d2_sentence))
        temp_list = get_max_common(d1_sentence, d2_sentence, MIN_COMMON_WORDS)
        if len(temp_list) != 0: 
            common_phrase_list.append(temp_list)
        
print(common_phrase_list)        

d1_doc:  hey~~~ there!hi, how are you now I am
fine, and you? I am good, too and I have never been better too. 

d2_doc:  hey~~~ there!hi, how are you now and I am
fine, and you? I am good, too and I have been better too. 

d1: hey there, d2: hey there
d1: hey there, d2: hi how are you now and I am fine and you
d1: hey there, d2: I am good too and I have been better too
d1: hi how are you now I am fine and you, d2: hey there
d1: hi how are you now I am fine and you, d2: hi how are you now and I am fine and you
[0, 5]:  hi how are you now
[5, 9]:  i am fine and
d1: hi how are you now I am fine and you, d2: I am good too and I have been better too
d1: I am good too and I have never been better too, d2: hey there
d1: I am good too and I have never been better too, d2: hi how are you now and I am fine and you
d1: I am good too and I have never been better too, d2: I am good too and I have been better too
[0, 7]:  i am good too and i have
[['hi how are you now', 'i am fine and'], ['i am goo